In [ ]:
import pandas as pd
import gc                         
gc.enable()

In [ ]:
orders = pd.read_csv('../input/instacart-market-basket-analysis/orders.csv' )
order_products_train = pd.read_csv('../input/instacart-market-basket-analysis/order_products__train.csv')
order_products_prior = pd.read_csv('../input/instacart-market-basket-analysis/order_products__prior.csv')
products = pd.read_csv('../input/instacart-market-basket-analysis/products.csv')
aisles = pd.read_csv('../input/instacart-market-basket-analysis/aisles.csv')
departments = pd.read_csv('../input/instacart-market-basket-analysis/departments.csv')

In [ ]:
orders = orders.loc[orders.user_id.isin(orders.user_id.drop_duplicates().sample(frac=0.1, random_state=25))] 

In [ ]:
orders.head()

In [ ]:
order_products_train.head()

In [ ]:
order_products_prior.head()

In [ ]:
products.head()

In [ ]:
aisles.head()

In [ ]:
departments.head()

In [ ]:
aisles['aisle'] = aisles['aisle'].astype('category')
departments['department'] = departments['department'].astype('category')
orders['eval_set'] = orders['eval_set'].astype('category')
products['product_name'] = products['product_name'].astype('category')

In [ ]:
op = orders.merge(order_products_prior, on='order_id', how='inner')
op.head()

In [ ]:
user = op.groupby('user_id')['order_number'].max().to_frame('u_total_orders')
user.head()

In [ ]:
user = user.reset_index()
user.head()

In [ ]:
u_reorder = op.groupby('user_id')['reordered'].mean().to_frame('u_reordered_ratio')
u_reorder = u_reorder.reset_index()
u_reorder.head()

In [ ]:
user = user.merge(u_reorder, on='user_id', how='left')

del u_reorder
gc.collect()

user.head()

In [ ]:
prd = op.groupby('product_id')['order_id'].count().to_frame('p_total_purchases')
prd.head()

In [ ]:
prd = prd.reset_index()
prd.head()

In [ ]:
p_reorder = op.groupby('product_id').filter(lambda x: x.shape[0] >40)
p_reorder.head()

In [ ]:
p_reorder = p_reorder.groupby('product_id')['reordered'].mean().to_frame('p_reorder_ratio')
p_reorder = p_reorder.reset_index()
p_reorder.head()

In [ ]:
prd = prd.merge(p_reorder, on='product_id', how='left')

del p_reorder
gc.collect()

prd.head()

In [ ]:
prd['p_reorder_ratio'] = prd['p_reorder_ratio'].fillna(value=0)
prd.head()

In [ ]:
uxp = op.groupby(['user_id', 'product_id'])['order_id'].count().to_frame('uxp_total_bought')
uxp.head()

In [ ]:
uxp = uxp.reset_index()
uxp.head()

In [ ]:
times = op.groupby(['user_id', 'product_id'])[['order_id']].count()
times.columns = ['Times_Bought_N']
times.head()

In [ ]:
total_orders = op.groupby('user_id')['order_number'].max().to_frame('total_orders')
total_orders.head()

In [ ]:
first_order_no = op.groupby(['user_id', 'product_id'])['order_number'].min().to_frame('first_order_number')
first_order_no  = first_order_no.reset_index()
first_order_no.head()

In [ ]:
span = pd.merge(total_orders, first_order_no, on='user_id', how='right')
span.head()

In [ ]:
span['Order_Range_D'] = span.total_orders - span.first_order_number + 1
span.head()

In [ ]:
uxp_ratio = pd.merge(times, span, on=['user_id', 'product_id'], how='left')
uxp_ratio.head()

In [ ]:
uxp_ratio['uxp_reorder_ratio'] = uxp_ratio.Times_Bought_N / uxp_ratio.Order_Range_D
uxp_ratio.head()

In [ ]:
uxp_ratio = uxp_ratio.drop(['Times_Bought_N', 'total_orders', 'first_order_number', 'Order_Range_D'], axis=1)
uxp_ratio.head()

In [ ]:
uxp_ratio['value'] = uxp_ratio.uxp_reorder_ratio > 0.25
uxp_ratio.head()

In [ ]:
del [times, first_order_no, span]

In [ ]:
uxp = uxp.merge(uxp_ratio, on=['user_id', 'product_id'], how='left')

del uxp_ratio
uxp.head()

In [ ]:
op[op.user_id==1].head(45)

In [ ]:
op['order_number_back'] = op.groupby('user_id')['order_number'].transform(max) - op.order_number +1 
op.head(15)

In [ ]:
op[op.user_id==7].head(10)

In [ ]:
op5 = op[op.order_number_back <= 5]
op5.head(15)

In [ ]:
last_five = op5.groupby(['user_id','product_id'])[['order_id']].count()
last_five.columns = ['times_last5']
last_five.head(10)

In [ ]:
uxp = uxp.merge(last_five, on=['user_id', 'product_id'], how='left')

del [op5 , last_five]
uxp.head()

In [ ]:
uxp = uxp.fillna(0)
uxp.head()

In [ ]:
data = uxp.merge(user, on='user_id', how='left')
data.head()

In [ ]:
data = data.merge(prd, on='product_id', how='left')
data.head()

In [ ]:
data.groupby('product_id').filter(lambda x: x.shape[0] >10)

In [ ]:
del op, user, prd, uxp
gc.collect()

In [ ]:
orders_future = orders[((orders.eval_set=='train') | (orders.eval_set=='test'))]
orders_future = orders_future[ ['user_id', 'eval_set', 'order_id'] ]
orders_future.head(10)

In [ ]:
data = data.merge(orders_future, on='user_id', how='left')
data.head(10)

In [ ]:
data_train = data[data.eval_set=='train']
data_train.head()

In [ ]:
data_train = data_train.merge(order_products_train[['product_id','order_id', 'reordered']], on=['product_id','order_id'], how='left' )
data_train.head(15)

In [ ]:
data_train['reordered'] = data_train['reordered'].fillna(0)
data_train.head(15)

In [ ]:
data_train = data_train.set_index(['user_id', 'product_id'])
data_train.head(15)

In [ ]:
data_train = data_train.drop(['eval_set', 'order_id'], axis=1)
data_train.head(15)